In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import datetime
import numpy as np


In [ ]:
daily = "https://www.tn.gov/health/cedep/ncov.html"
viral = pd.read_html(daily)
df=viral[3]
testing = df.drop([10])
age_groups = testing.rename(columns={'Age Ranges of Confirmed Cases.1': 'Number_Cases'})
age_groups


In [ ]:
daily = "https://www.tn.gov/health/cedep/ncov.html"
viral = pd.read_html(daily)
df2 = viral[5]
df2

In [18]:
# TNY='https://en.wikipedia.org/wiki/User:Michael_J/County_table'
# stuff=pd.read_html(TNY)
# TN_land_data=stuff[0]
# TN_land_data.to_csv('C:/Users/clayf/Documents/Coronavirus/Covid_predictions/TN/Resources/TN_land_data.csv')
land_data=pd.read_csv('C:/Users/clayf/Documents/Coronavirus/Covid_predictions/TN/Resources/TN_land_data.csv')
TN_lat_long=land_data[land_data['State'] == 'TN']

lat=TN_lat_long.iloc[:,13]
long=TN_lat_long.iloc[:,14]
lats=[]
longs=[]
for i in lat:
    x=i[:-1]
  
    
    lats.append(x)

for steve in long:
    y=steve[:-1]
  
    longs.append(y)
print(lats)


TN_lat_long['Latitude']=lats
TN_lat_long["Longitude"]=longs


TN_lat_long.reset_index(drop=True, inplace=True)
TN_lat_long.drop(["Unnamed: 0", "Sort [1]"],axis=1)
TN_lat_long["Region"] = 'US-TN'
TN_lat_long["Country"] = 'United States'
TN_lat_long

TN_lat_long.to_csv('C:/Users/clayf/Documents/Coronavirus/Covid_predictions/TN/Resources/TN_geo_data.csv', index=False)

['+36.116731', '+35.513660', '+36.069253', '+35.593668', '+35.688185', '+35.153914', '+36.401592', '+35.808394', '+35.967896', '+36.284744', '+36.255180', '+35.416639', '+36.501557', '+36.545765', '+35.916198', '+35.488759', '+35.811312', '+35.952398', '+36.169129', '+35.603422', '+35.982220', '+36.145533', '+36.054196', '+35.196993', '+36.376079', '+35.155926', '+35.991694', '+35.202723', '+36.277463', '+36.178998', '+35.387273', '+36.218397', '+35.159186', '+36.521420', '+35.218131', '+35.201893', '+36.452206', '+35.586690', '+35.653995', '+36.325398', '+35.802396', '+36.285777', '+36.040440', '+36.354242', '+36.048479', '+36.453204', '+35.992727', '+36.333905', '+35.762951', '+35.220476', '+35.523244', '+35.142532', '+35.737450', '+35.424471', '+35.175626', '+36.537838', '+35.606056', '+35.133422', '+35.468387', '+35.615696', '+35.503397', '+35.447666', '+36.500354', '+35.288889', '+36.138697', '+36.358175', '+36.344850', '+35.659785', '+36.559364', '+35.109437', '+36.140807', '+35.

C:\Users\clayf\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\clayf\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\clayf\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [ ]:
daily = "https://www.tn.gov/health/cedep/ncov.html"
viral = pd.read_html(daily)
df2 = viral[5]
TN_county = df2[:-3]
headers=TN_county.columns.values.tolist()
TN_county["County"]=TN_county["County"].str.split(" County", expand = True)
TN_county.columns = ['County', 'Positive', 'Negative', 'Death']
TN_county


In [ ]:
date = datetime.datetime.today()
date_modify=str(date)
date_to_add = date_modify[0:10]
TN_county["Date"]=date_to_add

In [ ]:
TNpop = pd.read_csv("Population Estimates by County.csv")
TNpop= TNpop.drop([96])
TNpop = TNpop.drop([0])
TNpop["Population Estimates by County"]= TNpop["Population Estimates by County"].str.split(" County", n = 1, expand = True)
TNpop = TNpop.rename(columns={'Population Estimates by County': 'County'})
TN_county_populations = TNpop.rename(columns={'Unnamed: 1': 'Population'})
TN_county_populations["Population"] = TN_county_populations["Population"].str.replace(",","").astype(int)
TN_county_populations


In [ ]:
TN_data = TN_county_populations.merge(TN_county,left_on='County',right_on='County', how='left')
TN_data["Percentage of County Population"] = round((TN_data["Positive"]/TN_data["Population"])*100,3)
TN_data=TN_data.sort_values("Positive", ascending=False).reset_index(drop=True)
TN_data["Percent_Tested"] = round(((TN_data["Positive"]+TN_data["Negative"])/TN_data["Population"])*100,1)
TN_data

In [ ]:
to_update=pd.read_csv("C:/Users/clayf/Documents/Coronavirus/Covid_predictions/TN/Resources/Synched/TN_cases_yesterday.csv")
synched=TN_data.append(to_update, sort=True)
TN_cases=synched.sort_values(["County", "Date"], ascending=True)
TN_cases["Duplicates"]=TN_cases["County"]+TN_cases['Date']
TN_cases = TN_cases.drop_duplicates(subset="Duplicates", keep="last")
TN_overtime=TN_cases.drop('Duplicates', axis=1)
TN_overtime=TN_overtime.drop('index', axis=1)
TN_overtime.reset_index(drop=True, inplace=True)
TN_overtime


In [ ]:
TN_overtime.sort_values(["County", "Date"], ascending=True)



## 

In [ ]:
TN_data["Percentage of County Population"] = round((TN_data["Positive"]/TN_data["Population"])*100,3)
TN_data.reset_index(drop=True)
TN_data["Percent_Tested"] = round(((TN_data["Positive"]+TN_data["Negative"])/TN_data["Population"])*100,1)
TN_data=TN_data.sort_values("Percent_Tested", ascending=False)
TN_data=TN_data
TN_data


In [ ]:
url2="https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
viral_times=pd.read_csv(url2)
TN=viral_times[viral_times["state"] == 'Tennessee']
# TN=TN[TN["date"] == '2020-03-31']
TN=TN.rename(columns={'county':'County'})
TN=TN.rename(columns={'date':'Date'})
TN=TN.rename(columns={'cases':'Positive'})
TN=TN.rename(columns={'deaths':'Deaths'})
TN=TN.drop(['state'], axis=1)
TN

In [ ]:
TN_deaths = TN_data.merge(TN,left_on='County',right_on='County', how='left')
TN_deaths.head(60)

df_counties=viral[4]
file = datetime.datetime.today()
date = file[0:10]
date


In [ ]:
# from pathlib import Path
stuff=pd.read_csv("C:/Users/clayf/Documents/VU-NSH-DATA-PT-07-2019-U-C/Activities/Week3/02/09-Ins_WriteCSV/output/new.csv")
stuff["crapola"]= 2
stuff.to_csv("C:/Users/clayf/Documents/VU-NSH-DATA-PT-07-2019-U-C/Activities/Week3/02/09-Ins_WriteCSV/output/new.csv", index=False)


In [ ]:
US = "https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html?CDC_AA_refVal=https%3A%2F%2Fwww.cdc.gov%2Fcoronavirus%2F2019-ncov%2Fcases-in-us.html"
us_data = pd.read_html(US)
us_total_cases = us_data[0]
us_total_cases.rename(columns={list(us_total_cases)[1]:'Total'}, inplace=True)
us_total_cases['Total']=us_total_cases['Total'].astype(int)
US = us_total_cases.iloc[3,1]

In [ ]:
date = datetime.datetime.today()
date_modify = str(date)
date_to_add = date_modify[0:10]
x['date'] = date_to_add



In [ ]:
TN_ratio = (TN/6700000)*100
US_ratio = (US/372000000)*100

print(TN_ratio)
print(US_ratio)